In [ ]:
import numpy as np
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [ ]:
from performStockAnalysis.main import select_emiten_to_model, develop_models_for_selected_emiten, forecast_using_the_developed_models

In [ ]:
selected_kode = select_emiten_to_model()

In [ ]:
develop_models_for_selected_emiten(['BBCA'], 'max_loss', [10, 15])

In [ ]:
performance_10dd = pd.read_csv('database/modelPerformances/maxLoss/10dd-20251027.csv') \
                        .dropna() \
                        .reset_index(drop=True) \
                        .sort_values('Threshold', ascending=False)

performance_15dd = pd.read_csv('database/modelPerformances/maxLoss/15dd-20251027.csv') \
                        .dropna() \
                        .reset_index(drop=True) \
                        .sort_values('Threshold', ascending=False)

In [ ]:
performance_10dd.describe()

In [ ]:
performance_15dd.describe()

In [ ]:
forecast_using_the_developed_models(10, 'maxLoss', '20251027', 0.5)

In [ ]:
forecast_using_the_developed_models(15, 'maxLoss', '20251027', 0.5)

In [ ]:
forecast_10_dd = pd.read_csv('database/forecastedStocks/maxLoss/forecast-10dd-20251027.csv')
forecast_15_dd = pd.read_csv('database/forecastedStocks/maxLoss/forecast-15dd-20251027.csv')

In [ ]:
pd.merge(performance_10dd, forecast_10_dd, on='Kode', how='inner').sort_values('Forecast Low Risk 10dd', ascending=False).head()

In [ ]:
pd.merge(performance_15dd, forecast_15_dd, on='Kode', how='inner').sort_values('Forecast Low Risk 15dd', ascending=False).head()

In [ ]:
forecast_10_dd.sort_values('Forecast Low Risk 10dd', ascending=False).head(20)

In [ ]:
forecast_15_dd.sort_values('Forecast Low Risk 15dd', ascending=False).head(20)